# Hometask

1) Find text to train (any book)<br>
2) Build train and validation set <br>
3) Train bidirectional language model that predicts the POS of word being based on its `n_context= 3` neighbours from the left and `n_context= 3` neighbours from the right <br>
4) Evaluate the model 

In [14]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.utils import to_categorical
import numpy as np

In [2]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lygov\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\lygov\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [3]:
sentences = brown.sents()
pos_tags = brown.tagged_sents(tagset='universal')

In [4]:
words = set(word for sent in sentences for word in sent)
word2id = {word: i+1 for i, word in enumerate(words)}
word2id['OOV'] = 0

In [6]:
tag2id = {tag: i for i, tag in enumerate(set(tag for sent in pos_tags for _, tag in sent))}

In [7]:
# Перетворюємо речення та теги в послідовності числових ID
X = [[word2id[word] if word in word2id else 0 for word in sent] for sent in sentences]
y = [[tag2id[tag] for _, tag in sent] for sent in pos_tags]

In [8]:
n_context = 3

In [9]:
X_new, y_new = [], []
for sent, tags in zip(X, y):
    for i in range(n_context, len(sent) - n_context):
        X_new.append(sent[i-n_context:i] + sent[i+1:i+n_context+1])
        y_new.append(tags[i])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_new), np.array(y_new), test_size=0.2)

In [16]:
y_train = to_categorical(y_train, num_classes=len(tag2id))
y_test = to_categorical(y_test, num_classes=len(tag2id))

In [17]:
model = Sequential()
model.add(Embedding(input_dim=len(word2id)+1, output_dim=64))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(len(tag2id), activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 1080s 52ms/step - accuracy: 0.5333 - loss: 1.4270 - val_accuracy: 0.6576 - val_loss: 1.0429
Epoch 2/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 1123s 54ms/step - accuracy: 0.7101 - loss: 0.8898 - val_accuracy: 0.6775 - val_loss: 0.9808
Epoch 3/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 927s 45ms/step - accuracy: 0.7700 - loss: 0.7129 - val_accuracy: 0.6785 - val_loss: 0.9989
Epoch 4/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 837s 40ms/step - accuracy: 0.8114 - loss: 0.5887 - val_accuracy: 0.6727 - val_loss: 1.0513
Epoch 5/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 838s 40ms/step - accuracy: 0.8425 - loss: 0.4945 - val_accuracy: 0.6649 - val_loss: 1.1235
Epoch 6/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 1253s 60ms/step - accuracy: 0.8651 - loss: 0.4251 - val_accuracy: 0.6535 - val_loss: 1.1954
Epoch 7/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 1589s 77ms/step - accuracy: 0.8816 - loss: 0.3729 - val_accuracy: 0.6477 - val_loss: 1.2998
Epoch 8/10
20713/20713 ━━━━━━━━━━━━━━━━━━━━ 3329s 1